In [2]:
import scrapy, requests
from scrapy.http import TextResponse

In [3]:
#!rm -rf covid_19
!scrapy startproject covid_19

New Scrapy project 'covid_19', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/covid_19/covid_19

You can start your first spider with:
    cd covid_19
    scrapy genspider example example.com


In [3]:
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}
req = requests.get('http://ncov.mohw.go.kr/',headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://ncov.mohw.go.kr/>

### 일일 확진자(국내, 해외유입, 합계)

In [141]:
date = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
country_in =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
country_out =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
total_country = [country_in+country_out for country_in,country_out in zip(country_in,country_out)]

date,country_in, country_out, total_country


(["(3.2. 00시 기준, '20.1.3. 이후 누계)"], [319], [25], [344])

### 검사 현황(누적 검사수, 누적 검사 완료수, 누적 확진률)

In [142]:
testing = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[1]/span[2]/text()').extract()
testing_finished = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[2]/span[2]/text()').extract()
percentage = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[3]/span[2]/text()').extract()
testing,testing_finished,percentage

(['6,681,976 건'], ['6,608,544 건'], ['1.4 %'])

In [6]:
%%writefile covid_19/covid_19/items.py
import scrapy


class Covid19Item(scrapy.Item):
    date = scrapy.Field()
    country_in = scrapy.Field()
    country_out = scrapy.Field()
    total_country = scrapy.Field()
    testing = scrapy.Field()
    testing_finished = scrapy.Field()
    percentage = scrapy.Field()

Overwriting covid_19/covid_19/items.py


In [53]:
%%writefile  covid_19/covid_19/spiders/spider.py
import scrapy,re
from covid_19.items import Covid19Item

class spider(scrapy.Spider):
    name = "Covid19"
    start_urls =  ["http://ncov.mohw.go.kr/"]
    
    def parse(self, response):  
#         links = response.xpath('/html/head/meta[8]/@content').extract()
#         for link in links:
            yield scrapy.Request("http://ncov.mohw.go.kr/", callback=self.parse_content)
   
    def parse_content(self,response):
        item = Covid19Item()
        item['date'] = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
        item['country_in'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
        item['country_out'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
        item['total_country'] = [item['country_in']+item['country_out'] for item['country_in'],item['country_out'] in zip(item['country_in'],item['country_out'])]
        item['testing'] = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[1]/span[2]/text()').extract()
        item['testing_finished'] = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[2]/span[2]/text()').extract()
        item['percentage'] = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[2]/div/div[1]/div[1]/div[1]/ul/li[3]/span[2]/text()').extract()
        yield item

Overwriting covid_19/covid_19/spiders/spider.py


In [8]:
%pwd

'/home/ubuntu/python3/notebook/covid_19'

In [9]:
%%writefile run.sh
cd /home/ubuntu/python3/notebook/covid_19/covid_19
scrapy crawl Covid19 -o covid_19_1.csv

Overwriting run.sh


In [10]:
!/bin/bash run.sh

2021-03-02 09:37:14 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: covid_19)
2021-03-02 09:37:14 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 25 2020, 07:37:56) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-02 09:37:14 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-02 09:37:14 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'covid_19',
 'NEWSPIDER_MODULE': 'covid_19.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['covid_19.spiders']}
2021-03-02 09:37:14 [scrapy.extensions.telnet] INFO: Telnet Password: 619e779a3aacb4da
2021-03-02 09:37:14 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 '

In [11]:
import pandas as pd
df = pd.read_csv('./covid_19/covid_19_1.csv')
df

,country_in,country_out,date,percentage,testing,testing_finished,total_country
0,338,17,"(3.1. 00시 기준, '20.1.3. 이후 누계)",1.4 %,"6,665,755 건","6,592,010 건",[355]
1,country_in,country_out,date,percentage,testing,testing_finished,total_country
2,338,17,"(3.1. 00시 기준, '20.1.3. 이후 누계)",1.4 %,"6,665,755 건","6,592,010 건",[355]
3,country_in,country_out,date,percentage,testing,testing_finished,total_country
4,319,25,"(3.2. 00시 기준, '20.1.3. 이후 누계)",1.4 %,"6,681,976 건","6,608,544 건",[344]
